# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS sparkifydatabase
        WITH REPLICATION = 
        {'class' : 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print (e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkifydatabase')
except Exception as e:
    print (e)

Query 1
Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

While creating the table we will use session_id and item_in_session combination as Primary Key as they make the record unique together.

In [8]:
create_table_1 = "CREATE TABLE IF NOT EXISTS song_details"
create_table_1 = create_table_1 + (""" (session_id INT, item_in_session INT, artist TEXT, song_title TEXT,
                                        song_length FLOAT, PRIMARY KEY (session_id, item_in_session)
                )""")
try:
    session.execute(create_table_1)
except Exception as e:
    print(e)
    
select_query_1 = """ SELECT artist, song_title, song_length FROM song_details
                    WHERE session_id = %s 
                    AND item_in_session = %s 
                    """

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_details(session_id, item_in_session, artist, song_title, song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

verifing that the data have been inserted into each table

In [10]:
try:
    rows = session.execute(select_query_1, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist, "\nSong Title: "+row.song_title, "\nSong Length: "+str(row.song_length))

Artist: Faithless 
Song Title: Music Matters (Mark Knight Dub) 
Song Length: 495.30731201171875


Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


While creating the table we will use user_id and  session_id and item_in_sesion combination as Primary Key and they partition the data.The column  item_in_session is used as clustering column that will sort the data in ascending order.

In [11]:
create_table_2 = "CREATE TABLE IF NOT EXISTS artist_details"
create_table_2 = create_table_2 + (""" (user_id INT, session_id INT, item_in_session INT, 
                                        artist_name TEXT, song_title TEXT, user_first_name TEXT, 
                                        user_last_name TEXT, PRIMARY KEY ((user_id, session_id), item_in_session)
                )""")
try:
    session.execute(create_table_2)
except Exception as e:
    print(e)
    
select_query_2 = """ SELECT artist_name, song_title, user_first_name, user_last_name FROM artist_details
                    WHERE user_id = %s
                    AND session_id = %s 
                    """
                    

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_details(user_id, session_id, item_in_session, artist_name, \
                                            song_title, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

                    

In [13]:
try:
    rows = session.execute(select_query_2, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print("Artist: "+row.artist_name, "\nSong Title: "+row.song_title, "\nUser First Name: "+row.user_first_name, "\nUser Last Name: "+row.user_last_name)

Artist: Down To The Bone 
Song Title: Keep On Keepin' On 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Three Drives 
Song Title: Greece 2000 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Sebastien Tellier 
Song Title: Kilometer 
User First Name: Sylvie 
User Last Name: Cruz
Artist: Lonnie Gordon 
Song Title: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) 
User First Name: Sylvie 
User Last Name: Cruz


Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

Here Song_title is used as partition key and the user_id is clustering column which will sort data in ascending order.The combination of both these columns makes the record unique.

In [14]:
create_table_3 = "CREATE TABLE IF NOT EXISTS user_details"
create_table_3 = create_table_3 + (""" (song_title TEXT, user_id INT, user_first_name TEXT, user_last_name TEXT, 
                                        PRIMARY KEY ((song_title), user_id)
                )""")
try:
    session.execute(create_table_3)
except Exception as e:
    print(e)

In [15]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_details(song_title, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [16]:
select_query_3 = """ SELECT user_first_name, user_last_name FROM user_details WHERE song_title = %s;"""


In [17]:
try:
    rows = session.execute(select_query_3, ('All Hands Against His Own', ))
except Exception as e:
    print(e)

for row in rows:
    print("User First Name: "+row.user_first_name, "\nUser Last Name: "+row.user_last_name)

User First Name: Jacqueline 
User Last Name: Lynch
User First Name: Tegan 
User Last Name: Levine
User First Name: Sara 
User Last Name: Johnson


### Drop the tables

In [18]:
session.execute("DROP TABLE IF EXISTS song_details")
session.execute("DROP TABLE IF EXISTS artist_details")
session.execute("DROP TABLE IF EXISTS user_details")

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()